In [1]:
# Import likely important packages, etc.
import sys, os, csv, re
from wcmatch import fnmatch
import datetime
import time
import xml.etree.ElementTree as et
from zipfile import ZipFile
import numpy as np
import pandas as pd
import xarray as xr
import shutil

In [2]:
from utils import *

In [3]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/CTDMO/CTDMO_Results'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/CTDMO/CTDMO_Cal'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/CTDMOG'

In [4]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [5]:
CTDMO = whoi_asset_tracking(excel_spreadsheet,sheet_name,instrument_class='CTDMO',whoi=True,series='G')
CTDMO

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
95,CTDMO,G,37-10214,115166,A00103,CGINS-CTDMOG-10214,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00254\n3305-00101-00569,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP02HYPM-00002\nGP03FLMA-00004,NaN,NaN,NaN
96,CTDMO,G,37-10215,115167\n118298,A00104,CGINS-CTDMOG-10215,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00404\n3305-00101-00745,NaN,NaN,3305-00900-00146\n3305-00900-00402,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMA-00001\nGP03FLMA-00003\nGP03FLMB-00005,NaN,NaN,NaN
97,CTDMO,G,37-10216,115168,A00105,CGINS-CTDMOG-10216,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00255\n3305-00101-00578,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00001\nGP03FLMB-00004,NaN,NaN,NaN
98,CTDMO,G,37-10217,115169\n118299,A00106,CGINS-CTDMOG-10217,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00399\n3305-00101-00738,NaN,NaN,3305-00900-00146\n3305-00900-00402,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00001\nGP03FLMA-00003\nGP03FLMB-00005,NaN,NaN,NaN
99,CTDMO,G,37-10218,115170,A00107,CGINS-CTDMOG-10218,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00256\n3305-00101-00559,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMA-00002\nGP03FLMB-00004,NaN,NaN,NaN
100,CTDMO,G,37-10219,115171\n118138,A00108,CGINS-CTDMOG-10219,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00436\n3305-00101-00671,NaN,NaN,3305-00900-00152\n3305-00900-00392,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMA-00001\nGP03FLMA-00003\nGI02HYPM-00004,NaN,NaN,NaN
101,CTDMO,G,37-10220,115172\n118300,A00109,CGINS-CTDMOG-10220,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00403\n3305-00101-00722,NaN,NaN,3305-00900-00146\n3305-00900-00402,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMA-00001\nGP03FLMA-00003\nGP03FLMB-00005,NaN,NaN,NaN
102,CTDMO,G,37-10221,115173\n118301,A00110,CGINS-CTDMOG-10221,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00426\n3305-00101-00740,NaN,NaN,3305-00900-00146\n3305-00900-00402,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00001\nGP03FLMA-00003\nGP03FLMB-00005,NaN,NaN,NaN
103,CTDMO,G,37-10222,115174,A00111,CGINS-CTDMOG-10222,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00257\n3305-00101-00588,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00001\nGP03FLMB-00004,NaN,NaN,NaN
104,CTDMO,G,37-10223,115175,A00112,CGINS-CTDMOG-10223,37IM,1336-00001-00007,3.1,SeaBird,...,3305-00101-00258\n3305-00101-00583,NaN,NaN,3305-00900-00060\n3305-00900-00257,WH-SC11-01-CTD-1004,12/12/2012,GP03FLMB-00002\nGP03FLMB-00004,NaN,NaN,NaN


In [ ]:
list(set(CTDMO['Series']))

In [6]:
def get_qct_files(df, qct_directory):
    qct_dict = {}
    uids = list(set(df['UID']))
    for uid in uids:
        df['UID_match'] = df['UID'].apply(lambda x: True if uid in x else False)
        qct_series = df[df['UID_match'] == True]['QCT Testing']
        qct_series = list(qct_series.iloc[0].split('\n'))
        qct_dict.update({uid:qct_series})
    return qct_dict

In [7]:
def get_calibration_files(serial_nums, cal_directory):
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)
        sn = str(sn[:])
        files = []
        for file in os.listdir(cal_directory):
            if 'Calibration_File' in file:
                if sn in file:
                    files.append(file)
        calibration_files.update({uid:files})
    return calibration_files

In [8]:
# Now I need to load the all of the csv files based on their UID
def load_csv_info(csv_dict,filepath):
    """
    Loads the calibration coefficient information contained in asset management
    
    Args:
        csv_dict - a dictionary which associates an instrument UID to the
            calibration csv files in asset management
        filepath - the path to the directory containing the calibration csv files
    Returns:
        csv_cals - a dictionary which associates an instrument UID to a pandas
            dataframe which contains the calibration coefficients. The dataframes
            are indexed by the date of calibration
    """
    
    # Load the calibration data into pandas dataframes, which are then placed into
    # a dictionary by the UID
    csv_cals = {}
    for uid in csv_dict:
        cals = pd.DataFrame()
        for file in csv_dict[uid]:
            data = pd.read_csv(filepath+file)
            date = file.split('__')[1].split('.')[0]
            data['CAL DATE'] = pd.to_datetime(date)
            cals = cals.append(data)
        csv_cals.update({uid:cals})
        
    # Pivot the dataframe to be sorted based on calibration date
    for uid in csv_cals:
        csv_cals[uid] = csv_cals[uid].pivot(index=csv_cals[uid]['CAL DATE'], columns='name')['value']
        
    return csv_cals

In [11]:
qct_dict = get_qct_files(CTDMO, qct_directory)
qct_dict

{'CGINS-CTDMOG-12407': ['3305-00101-00043',
  '3305-00101-00445',
  '3305-00101-00621'],
 'CGINS-CTDMOG-13422': ['3305-00101-00148',
  '3305-00101-00474',
  '3305-00101-00721'],
 'CGINS-CTDMOG-13598': ['3305-00101-00201', '3305-00101-00529'],
 'CGINS-CTDMOG-12313': ['3305-00101-00043',
  '3305-00101-00408',
  '3305-00101-00648'],
 'CGINS-CTDMOG-11595': ['3305-00101-00271', '3305-00101-00587'],
 'CGINS-CTDMOG-12314': ['3305-00101-00043',
  '3305-00101-00409',
  '3305-00101-00632'],
 'CGINS-CTDMOG-13435': ['3305-00101-00159',
  '3305-00101-00466',
  '3305-00101-00701',
  '3305-00101-00744'],
 'CGINS-CTDMOG-10254': ['3305-00101-00401', '3305-00101-00728'],
 'CGINS-CTDMOG-12311': ['3305-00101-00043', '3305-00101-00406'],
 'CGINS-CTDMOG-13432': ['3305-00101-00156',
  '3305-00101-00469',
  '3305-00101-00712'],
 'CGINS-CTDMOG-12375': ['3305-00101-00043',
  '3305-00101-00420',
  '3305-00101-00670'],
 'CGINS-CTDMOG-12396': ['3305-00101-00043',
  '3305-00101-00437',
  '3305-00101-00638'],
 'CGIN

In [12]:
csv_dict = load_asset_management(CTDMO, asset_management_directory)
csv_dict

{'CGINS-CTDMOG-10214': ['CGINS-CTDMOG-10214__20170829.csv',
  'CGINS-CTDMOG-10214__20121113.csv',
  'CGINS-CTDMOG-10214__20140214.csv',
  'CGINS-CTDMOG-10214__20151110.csv'],
 'CGINS-CTDMOG-10215': ['CGINS-CTDMOG-10215__20160915.csv',
  'CGINS-CTDMOG-10215__20121113.csv',
  'CGINS-CTDMOG-10215__20181009.csv'],
 'CGINS-CTDMOG-10216': ['CGINS-CTDMOG-10216__20170903.csv',
  'CGINS-CTDMOG-10216__20121113.csv',
  'CGINS-CTDMOG-10216__20151113.csv'],
 'CGINS-CTDMOG-10217': ['CGINS-CTDMOG-10217__20160831.csv',
  'CGINS-CTDMOG-10217__20121114.csv',
  'CGINS-CTDMOG-10217__20181009.csv'],
 'CGINS-CTDMOG-10218': ['CGINS-CTDMOG-10218__20151108.csv',
  'CGINS-CTDMOG-10218__20170901.csv',
  'CGINS-CTDMOG-10218__20121115.csv',
  'CGINS-CTDMOG-10218__20140214.csv'],
 'CGINS-CTDMOG-10219': ['CGINS-CTDMOG-10219__20160921.csv',
  'CGINS-CTDMOG-10219__20180818.csv',
  'CGINS-CTDMOG-10219__20121114.csv'],
 'CGINS-CTDMOG-10220': ['CGINS-CTDMOG-10220__20121114.csv',
  'CGINS-CTDMOG-10220__20160827.csv',
  'C

In [15]:
uids = list(set(CTDMO['UID']))

In [16]:
serial_nums = get_serial_nums(CTDMO, uids)

In [17]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

{'CGINS-CTDMOG-12407': ['CTDMO-G_SBE_37IM_SN_37-12407_Calibration_Files_2015-10-27.pdf',
  'CTDMO-G_SBE_37IM_SN_37-12407_Calibration_Files_2016-06-23.zip',
  'CTDMO-G_SBE_37IM_SN_37-12407_Calibration_Files_2018-04-03.zip'],
 'CGINS-CTDMOG-13422': ['CTDMO-G_SBE_37IM_SN_37-13422_Calibration_Files_2015-06-02.pdf',
  'CTDMO-G_SBE_37IM_SN_37-13422_Calibration_Files_2016-10-15.zip',
  'CTDMO-G_SBE_37IM_SN_37-13422_Calibration_Files_2018-10-02.zip'],
 'CGINS-CTDMOG-13598': ['CTDMO-G_SBE_37IM_SN_37-13598_Calibration_Files_2015-07-01.pdf',
  'CTDMO-G_SBE_37IM_SN_37-13598_Calibration_Files_2017-04-03.zip'],
 'CGINS-CTDMOG-12313': ['CTDMO-G_SBE_37IM_SN_37-12313_Calibration_Files_2015-10-27.pdf',
  'CTDMO-G_SBE_37IM_SN_37-12313_Calibration_Files_2018-04-03.zip',
  'CTDMO-Q_SBE_37IM_SN_37-12313_Calibration_Files_2016-06-22.zip'],
 'CGINS-CTDMOG-11595': ['CTDMO-G_SBE_37IM_SN_37-11595_Calibration_Files_2014-01-25.pdf',
  'CTDMO-G_SBE_37IM_SN_37-11595_Calibration_Files_2016-01-13.pdf',
  'CTDMO-G_SBE_

In [19]:
uid = uids[0]
uid

'CGINS-CTDMOG-12407'

In [234]:
sorted(cal_dict[uid])

['CTDMO-G_SBE_37IM_SN_37-10216_Calibration_Files_2012-11-13.pdf',
 'CTDMO-G_SBE_37IM_SN_37-10216_Calibration_Files_2016-01-13.pdf',
 'CTDMO-G_SBE_37IM_SN_37-10216_Calibration_Files_2017-09-03.zip']

In [235]:
sorted(csv_dict[uid])

['CGINS-CTDMOG-10216__20121113.csv',
 'CGINS-CTDMOG-10216__20151113.csv',
 'CGINS-CTDMOG-10216__20170903.csv']

In [236]:
csv_path = generate_file_path(asset_management_directory, sorted(csv_dict[uid])[0])
csv_path

'/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/CTDMOG/CGINS-CTDMOG-10216__20121113.csv'

In [237]:
pdf_path = generate_file_path(cal_directory, sorted(cal_dict[uid])[0])
pdf_path

'/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/CTDMO/CTDMO_Cal/CTDMO-G_SBE_37IM_SN_37-10216_Calibration_Files_2012-11-13.pdf'

In [238]:
ensure_dir('/'.join((os.getcwd(),'temp','pdf')))

In [242]:
shutil.rmtree('/'.join((os.getcwd(),'temp','pdf')))

In [243]:
if pdf_path.endswith('.zip'):
    with ZipFile(pdf_path,'r') as zfile:
        for file in zfile.namelist():
            dirfile = file.split('/')[0]
            zfile.extract(file,path='/'.join((os.getcwd(),'temp','pdf')))
else:
    ensure_dir('/'.join((os.getcwd(),'temp','pdf')))
    shutil.copy(pdf_path, '/'.join((os.getcwd(),'temp','pdf')))


In [244]:
os.listdir('/'.join((os.getcwd(),'temp','pdf')))

['CTDMO-G_SBE_37IM_SN_37-10216_Calibration_Files_2012-11-13.pdf']

In [25]:
CTD = CTDCalibration(uid)

In [34]:
dirfile = 'CTDMO-G_SBE_37IM_SN_37-10216_Calibration_Files_2017-09-03'

In [37]:
# Try using textract to get the pdf text
import textract

In [248]:
pdf = textract.process(temp_path+'/CTDMO-G_SBE_37IM_SN_37-10216_Calibration_Files_2012-11-13.pdf', method='pdfminer', language='eng', encoding='ascii')
pdf = pdf.decode('ascii')

In [251]:
for line in pdf.splitlines():
    
    if 'SERIAL NUMBER' in line:
        *ignore, sn = line.split(':')
        
    if 'CALIBRATION DATE' in line:
        *ignore, cal_date = line.split(':')
        cal_date = pd.to_datetime(cal_date).strftime('%Y%m%d')
        CTD.date.update({len(CTD.date): cal_date})
        
    if any([word for word in line.split() if word.lower() in CTD.mo_coefficient_name_map.keys()]):
        

Sea-Bird Electronics, Inc. 

Advancing the Science of Ocean Measurement 

 

SBE37-IM MicroCAT

Conductivity, Temperature and (optional) Pressure Recorder
with Inductive Modem

Instrument Configuration

Instrument Serial Number:
Instrument Firmware Version:
IM Firmware Version
Zero Conductivity Frequency:
Instrument ID#:

Communications Format:
Baud Rate:
Data Bits:
Parity Bit:

Pressure Sensor Serial Number:
Pressure Sensor Manufacturer:
Pressure Sensor Range

Maximum Depth:

37-10216
3.1
2.0a
2560.582
16

IM
9600
8
None

3713002
Druck
1000db

1000m

CAUTION - The maximum deployment depth will be limited by the measurement
range of the optional pressure sensor, if installed, or the housing

Support Telephone: 

(+1)425-643-9866 

Support Email:   

seabird@seabird.com 


Sea-Bird Electronics, Inc.

13431 NE 20th Street, Bellevue, WA 98005-2010 USA
Phone: (+1) 425-643-9866 Fax (+1) 425-643-9954 Email: seabird@seabird.com

 
SENSOR SERIAL NUMBER: 10216
CALIBRATION DATE: 13-Nov-12
 
ITS-

In [226]:
# Add all the different pieces into a single wrapping function

def mo_parse_pdf(temp_path):
    text = {}
    for file in os.listdir(temp_path):
        # Read the pdf using OCR
        filepath = '/'.join((temp_path, file))
        pdf = textract.process(filepath, method='pdfminer', language='eng', encoding='ascii')
        # Now convert the extracted pdf text to ASCII
        pdf = pdf.decode('ascii')
        # Save the text in the dictionary
        text.update({file: pdf})
        
    return text


def mo_parse_pressure_cal(text):
    
    values = []
    coeffs = []
    flag = False
    for item in text.split('\n'):
    
        if 'COEFFICIENTS' in item:
            flag = True
            continue
    
        if 'SPAN' in item and flag is True:
            flag = False
        
        if 'S/N' in item:
            prange = item.split()[0]
            coeffs.append('prange')
            values.append(prange)
        
        if flag is True and len(item) > 0:
            data = item.split()
            try:
                float(data[0])
                values.append(data[0].strip())
            except:
                coeffs.append(data[0].strip())
            
    for n, k in enumerate(coeffs):
        name = CTD.mo_coefficient_name_map[k.lower()]
        CTD.coefficients.update({name: values[n]})
        
        
def mo_parse_ts_cal(text):
    keys = list(CTD.mo_coefficient_name_map.keys())

    for item in text.split('\n'):
    
        # Get the serial number
        if 'SERIAL NUMBER' in item:
            *ignore, sn = item.split(':')
        
        # Get the calibration date    
        if 'CALIBRATION DATE' in item:
            *ignore, cal_date = item.split(':')
            cal_date = pd.to_datetime(cal_date).strftime('%Y%m%d')
            CTD.date.update({len(CTD.date):cal_date})
        
        # Get the coefficient names and values
        if any([word for word in item.strip().split() if word.lower() in keys]):
            # Eliminate the returned lines that are equations
            try:
                coeff, value = item.split('=')
                float(value)
                print(coeff, value)
                name = CTD.mo_coefficient_name_map[coeff.lower().strip()]
                CTD.coefficients.update({name: value.strip()})
            except:
                pass

    
def mo_load_pdf(temp_path):
    
    text = parse_pdf(temp_path)
    
    for k in text.keys():
        if 'P'+CTD.serial in k:
            parse_pressure_cal(text[k])
            
        else:
            parse_ts_cal(text[k])
            
        

In [221]:
CTD = CTDCalibration(uid)

In [223]:
load_pdf(temp_path)

a0   -9.187993e-005
a1    3.077371e-004
a2   -4.669244e-006
a3    2.062453e-007
g   -9.838870e-001
h    1.507186e-001
i   -3.842321e-004
j    5.045918e-005
CPcor   -9.5700e-008
CTcor    3.2500e-006
WBOTC    6.5090e-007


In [225]:
CTD.date

{0: '20170903', 1: '20170903'}

In [224]:
CTD.coefficients

{'CC_p_range': '1450',
 'CC_pa0': '-2.249954e-002',
 'CC_pa1': '4.858358e-003',
 'CC_pa2': '-1.460866e-011',
 'CC_ptempa0': '-6.792154e+001',
 'CC_ptempa1': '5.204997e-002',
 'CC_ptempa2': '-4.593367e-007',
 'CC_ptca0': '5.247360e+005',
 'CC_ptca1': '1.561328e+000',
 'CC_ptca2': '-7.182982e-002',
 'CC_ptcb0': '2.537525e+001',
 'CC_ptcb1': '-1.350000e-003',
 'CC_ptcb2': '0.000000e+000',
 'CC_a0': '-9.187993e-005',
 'CC_a1': '3.077371e-004',
 'CC_a2': '-4.669244e-006',
 'CC_a3': '2.062453e-007',
 'CC_g': '-9.838870e-001',
 'CC_h': '1.507186e-001',
 'CC_i': '-3.842321e-004',
 'CC_j': '5.045918e-005',
 'CC_cpcor': '-9.5700e-008',
 'CC_ctcor': '3.2500e-006',
 'CC_wbotc': '6.5090e-007'}

In [205]:
k

'PTCB2'

In [209]:
uid = 'CGINS-CTDMOG-10216'

In [206]:
text.keys()

dict_keys(['SBE 37 V2 P10216 28Aug17.pdf', 'SBE 37 V2 T10216 03Sep17.pdf', 'SBE-37-10216-PressureTest.pdf', 'SBE 37 V2 C10216 03Sep17.pdf'])

In [136]:
text = {}
for file in os.listdir(temp_path):
    # Read the pdf using OCR
    filepath = '/'.join((temp_path, file))
    pdf = textract.process(filepath, method='pdfminer', language='eng', encoding='ascii')
    # Now convert the extracted pdf text to ASCII
    pdf = pdf.decode('ascii')
    # Save the text in the dictionary
    text.update({file: pdf})
    

In [187]:
# This parses the Pressure calibration document, which needs to be handled differently than the conductivity
# and temperature calibration documents
values = []
coeffs = []
flag = False
for item in text['SBE 37 V2 P10216 28Aug17.pdf'].split('\n'):
    
    if 'COEFFICIENTS' in item:
        flag = True
        continue
    
    if 'SPAN' in item and flag is True:
        flag = False
        
    if 'S/N' in item:
        prange = item.split()[0]
        coeffs.append('prange')
        values.append(prange)
        
    if flag is True and len(item) > 0:
        data = item.split()
        try:
            float(data[0])
            values.append(data[0].strip())
        except:
            coeffs.append(data[0].strip())
            
for n, k in enumerate(coeffs):
    name = CTD.mo_coefficient_name_map[k.lower()]
    CTD.coefficients.update({name: values[n]})
    
CTD.coefficients

{'CC_pa0': '-2.249954e-002',
 'CC_pa1': '4.858358e-003',
 'CC_pa2': '-1.460866e-011',
 'CC_ptempa0': '-6.792154e+001',
 'CC_ptempa1': '5.204997e-002',
 'CC_ptempa2': '-4.593367e-007',
 'CC_ptca0': '5.247360e+005',
 'CC_ptca1': '1.561328e+000',
 'CC_ptca2': '-7.182982e-002',
 'CC_ptcb0': '2.537525e+001',
 'CC_ptcb1': '-1.350000e-003',
 'CC_ptcb2': '0.000000e+000',
 'CC_a0': ' -9.187993e-005',
 'CC_a1': '  3.077371e-004',
 'CC_a2': ' -4.669244e-006',
 'CC_a3': '  2.062453e-007',
 'CC_g': ' -9.838870e-001',
 'CC_h': '  1.507186e-001',
 'CC_i': ' -3.842321e-004',
 'CC_j': '  5.045918e-005',
 'CC_cpcor': ' -9.5700e-008',
 'CC_ctcor': '  3.2500e-006',
 'CC_wbotc': '  6.5090e-007',
 'CC_p_range': '1450'}

In [185]:
CTD.mo_coefficient_name_map

{'ptcb1': 'CC_ptcb1',
 'pa2': 'CC_pa2',
 'a3': 'CC_a3',
 'pa0': 'CC_pa0',
 'wbotc': 'CC_wbotc',
 'ptcb0': 'CC_ptcb0',
 'g': 'CC_g',
 'ptempa1': 'CC_ptempa1',
 'ptcb2': 'CC_ptcb2',
 'a0': 'CC_a0',
 'h': 'CC_h',
 'ptca0': 'CC_ptca0',
 'a2': 'CC_a2',
 'cpcor': 'CC_cpcor',
 'i': 'CC_i',
 'ptempa0': 'CC_ptempa0',
 'prange': 'CC_p_range',
 'ctcor': 'CC_ctcor',
 'a1': 'CC_a1',
 'j': 'CC_j',
 'ptempa2': 'CC_ptempa2',
 'pa1': 'CC_pa1',
 'ptca1': 'CC_ptca1',
 'ptca2': 'CC_ptca2'}

In [167]:
CTD = CTDCalibration(uid=uid)

In [190]:
keys = list(CTD.mo_coefficient_name_map.keys())

for item in text['SBE 37 V2 T10216 03Sep17.pdf'].split('\n'):
    
    # Get the serial number
    if 'SERIAL NUMBER' in item:
        *ignore, sn = item.split(':')
        
    # Get the calibration date    
    if 'CALIBRATION DATE' in item:
        *ignore, cal_date = item.split(':')
        
    # Get the coefficient names and values
    if any([word for word in item.strip().split() if word.lower() in keys]):
        # Eliminate the returned lines that are equations
        try:
            coeff, value = item.split('=')
            float(value)
            print(coeff, value)
            name = CTD.mo_coefficient_name_map[coeff.lower().strip()]
            CTD.coefficients.update({name: value.strip()})
        except:
            pass    

a0   -9.187993e-005
a1    3.077371e-004
a2   -4.669244e-006
a3    2.062453e-007


In [191]:
CTD.coefficients

{'CC_pa0': '-2.249954e-002',
 'CC_pa1': '4.858358e-003',
 'CC_pa2': '-1.460866e-011',
 'CC_ptempa0': '-6.792154e+001',
 'CC_ptempa1': '5.204997e-002',
 'CC_ptempa2': '-4.593367e-007',
 'CC_ptca0': '5.247360e+005',
 'CC_ptca1': '1.561328e+000',
 'CC_ptca2': '-7.182982e-002',
 'CC_ptcb0': '2.537525e+001',
 'CC_ptcb1': '-1.350000e-003',
 'CC_ptcb2': '0.000000e+000',
 'CC_a0': '-9.187993e-005',
 'CC_a1': '3.077371e-004',
 'CC_a2': '-4.669244e-006',
 'CC_a3': '2.062453e-007',
 'CC_g': '-9.838870e-001',
 'CC_h': '1.507186e-001',
 'CC_i': '-3.842321e-004',
 'CC_j': '5.045918e-005',
 'CC_cpcor': '-9.5700e-008',
 'CC_ctcor': '3.2500e-006',
 'CC_wbotc': '6.5090e-007',
 'CC_p_range': '1450'}

In [80]:
CTD.mo_coefficient_name_map

{'ptcb1': 'CC_ptcb1',
 'pa2': 'CC_pa2',
 'a3': 'CC_a3',
 'pa0': 'CC_pa0',
 'wbotc': 'CC_wbotc',
 'ptcb0': 'CC_ptcb0',
 'g': 'CC_g',
 'ptempa1': 'CC_ptempa1',
 'ptcb2': 'CC_ptcb2',
 'a0': 'CC_a0',
 'h': 'CC_h',
 'ptca0': 'CC_ptca0',
 'a2': 'CC_a2',
 'cpcor': 'CC_cpcor',
 'i': 'CC_i',
 'ptempa0': 'CC_ptempa0',
 'prange': 'CC_p_range',
 'ctcor': 'CC_ctcor',
 'a1': 'CC_a1',
 'j': 'CC_j',
 'ptempa2': 'CC_ptempa2',
 'pa1': 'CC_pa1',
 'ptca1': 'CC_ptca1',
 'ptca2': 'CC_ptca2'}

In [ ]:
CTD.read_pdf(filepath)

In [ ]:
CTD.load_pdf(temp_path+'/'+'SBE 37 V2 P10216 28Aug17.pdf')

In [ ]:
filepath

In [ ]:
CTD.coefficients

In [ ]:
text

In [ ]:
CTD.read_pdf(pdf_path)

In [ ]:
CTD.coefficients

In [9]:
def generate_file_path(dirpath, filename, ext=['.cap', '.txt', '.log'], exclude=['_V', '_Data_Workshop']):
    """
    Function which searches for the location of the given file and returns
    the full path to the file.

    Args:
        dirpath - parent directory path under which to search
        filename - the name of the file to search for
        ext - file endings to search for
        exclude - optional list which allows for excluding certain
            directories from the search
    Returns:
        fpath - the file path to the filename from the current
            working directory.
    """
    # Check if the input file name has an extension already
    # If it does, parse it for input into the search algo
    if '.' in filename:
        check = filename.split('.')
        filename = check[0]
        ext = ['.'+check[1]]

    for root, dirs, files in os.walk(dirpath):
        dirs[:] = [d for d in dirs if d not in exclude]
        for fname in files:
            if fnmatch.fnmatch(fname, [filename+'*'+x for x in ext]):
                fpath = os.path.join(root, fname)
                return fpath

In [10]:
import datetime
import re
import os
from wcmatch import fnmatch
import xml.etree.ElementTree as et
import pandas as pd
import numpy as np
import string
from zipfile import ZipFile
import csv
import PyPDF2
from nltk.tokenize import word_tokenize
import json

class CTDCalibration():
    # Class that stores calibration values for CTDs.

    def __init__(self, uid):
        self.serial = ''
        self.uid = uid
        self.ctd_type = uid
        self.coefficients = {}
        self.date = {}

        self.coefficient_name_map = {
            'TA0': 'CC_a0',
            'TA1': 'CC_a1',
            'TA2': 'CC_a2',
            'TA3': 'CC_a3',
            'CPCOR': 'CC_cpcor',
            'CTCOR': 'CC_ctcor',
            'CG': 'CC_g',
            'CH': 'CC_h',
            'CI': 'CC_i',
            'CJ': 'CC_j',
            'G': 'CC_g',
            'H': 'CC_h',
            'I': 'CC_i',
            'J': 'CC_j',
            'PA0': 'CC_pa0',
            'PA1': 'CC_pa1',
            'PA2': 'CC_pa2',
            'PTEMPA0': 'CC_ptempa0',
            'PTEMPA1': 'CC_ptempa1',
            'PTEMPA2': 'CC_ptempa2',
            'PTCA0': 'CC_ptca0',
            'PTCA1': 'CC_ptca1',
            'PTCA2': 'CC_ptca2',
            'PTCB0': 'CC_ptcb0',
            'PTCB1': 'CC_ptcb1',
            'PTCB2': 'CC_ptcb2',
            # additional types for series O
            'C1': 'CC_C1',
            'C2': 'CC_C2',
            'C3': 'CC_C3',
            'D1': 'CC_D1',
            'D2': 'CC_D2',
            'T1': 'CC_T1',
            'T2': 'CC_T2',
            'T3': 'CC_T3',
            'T4': 'CC_T4',
            'T5': 'CC_T5',
        }

        # Name mapping for the MO-type CTDs (when reading from pdfs)
        self.mo_coefficient_name_map = {
            'ptcb1': 'CC_ptcb1',
            'pa2': 'CC_pa2',
            'a3': 'CC_a3',
            'pa0': 'CC_pa0',
            'wbotc': 'CC_wbotc',
            'ptcb0': 'CC_ptcb0',
            'g': 'CC_g',
            'ptempa1': 'CC_ptempa1',
            'ptcb2': 'CC_ptcb2',
            'a0': 'CC_a0',
            'h': 'CC_h',
            'ptca0': 'CC_ptca0',
            'a2': 'CC_a2',
            'cpcor': 'CC_cpcor',
            'i': 'CC_i',
            'ptempa0': 'CC_ptempa0',
            'prange': 'CC_p_range',
            'ctcor': 'CC_ctcor',
            'a1': 'CC_a1',
            'j': 'CC_j',
            'ptempa2': 'CC_ptempa2',
            'pa1': 'CC_pa1',
            'ptca1': 'CC_ptca1',
            'ptca2': 'CC_ptca2',
        }

        self.o2_coefficients_map = {
            'A': 'CC_residual_temperature_correction_factor_a',
            'B': 'CC_residual_temperature_correction_factor_b',
            'C': 'CC_residual_temperature_correction_factor_c',
            'E': 'CC_residual_temperature_correction_factor_e',
            'SOC': 'CC_oxygen_signal_slope',
            'OFFSET': 'CC_frequency_offset'
        }

    @property
    def uid(self):
        return self._uid

    @uid.setter
    def uid(self, d):
        r = re.compile('.{5}-.{6}-.{5}')
        if r.match(d) is not None:
            self.serial = d.split('-')[2]
            self._uid = d
        else:
            raise Exception(f"The instrument uid {d} is not a valid uid. Please check.")

    @property
    def ctd_type(self):
        return self._ctd_type

    @ctd_type.setter
    def ctd_type(self, d):
        if 'MO' in d:
            self._ctd_type = '37'
        elif 'BP' in d:
            self._ctd_type = '16'
        else:
            self._ctd_type = ''

    def load_pdf(self, filepath):
        """
        This function opens and loads a pdf into a parseable format.

        Args:
            filepath - full directory path with filename
        Raises:
            IOError - error reading or loading text from the pdf object
        Returns:
            text - a dictionary with page numbers as keys and the pdf text as items
        """

        # Open and read the pdf file
        pdfFileObj = open(filepath, 'rb')
        # Create a reader to be parsed
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        # Now, enumerate through the pdf and decode the text
        num_pages = pdfReader.numPages
        count = 0
        text = {}

        while count < num_pages:
            pageObj = pdfReader.getPage(count)
            count = count + 1
            text.update({count: pageObj.extractText()})

        # Run a check that text was actually extracted
        if len(text) == 0:
            raise(IOError(f'No text was parsed from the pdf file {filepath}'))
        else:
            return text

    def read_pdf(self, filepath):
        """
        Function which parses the opened and loaded pdf file into the
        relevant calibration coefficient data. This function works if
        the calibration pdfs have been split based on sensor as well as
        for combined pdfs.

        Args:
            text - the opened and loaded pdf text returned from load_pdf
        Raises:
            Exception - thrown when a relevant calibration information is
                missing from the text
        Returns:
            date - the calibration dates of the temperature, conductivity,
                and pressure sensors of the CTDMO in a dictionary object
            serial - populated serial number of the CTDMO
            coefficients - populated dictionary of the calibration coefficients
                as keys and associated values as items.
        """
        text = self.load_pdf(filepath)

        for page_num in text.keys():
            # Search for the temperature calibration data
            if 'SBE 37 TEMPERATURE CALIBRATION DATA' in text[page_num]:
                tokens = word_tokenize(text[page_num])
                data = [word.lower() for word in tokens if not word in string.punctuation]
                # Now, find and record the calibration date
                if 'calibration' and 'date' in data:
                    cal_ind = data.index('calibration')
                    date_ind = data.index('date')
                    # Run check they are in order
                    if date_ind == cal_ind+1:
                        date = pd.to_datetime(data[date_ind+1]).strftime('%Y%m%d')
                        self.date.update({'TCAL':date})
                    else:
                        raise Exception(f"Can't locate temp calibration date.")
                else:
                    raise Exception(f"Can't locate temp calibration date.")

                # Check for the serial number
                if 'serial' and 'number' in data and len(self.serial) == 0:
                    ser_ind = data.index('serial')
                    num_ind = data.index('number')
                    if num_ind == ser_ind+1:
                        self.serial = data[num_ind+1]
                    else:
                        pass

                # Now, get the calibration coefficients
                for key in self.mo_coefficient_name_map.keys():
                    if key in data:
                        ind = data.index(key)
                        self.coefficients.update({self.mo_coefficient_name_map[key]: data[ind+1]})
                    else:
                        pass

            # Search for the conductivity calibration data
            elif 'SBE 37 CONDUCTIVITY CALIBRATION DATA' in text[page_num]:
                # tokenize the text data and extract only key words
                tokens = word_tokenize(text[page_num])
                data = [word.lower() for word in tokens if not word in string.punctuation]

                # Now, find and record the calibration date
                if 'calibration' and 'date' in data:
                    cal_ind = data.index('calibration')
                    date_ind = data.index('date')
                    # Run check they are in order
                    if date_ind == cal_ind+1:
                        date = pd.to_datetime(data[date_ind+1]).strftime('%Y%m%d')
                        self.date.update({'CCAL': date})
                    else:
                        raise Exception(f"Can't locate conductivity calibration date.")
                else:
                    raise Exception(f"Can't locate conductivity calibration date.")

                # Check for the serial number
                if 'serial' and 'number' in data and len(self.serial) == 0:
                    ser_ind = data.index('serial')
                    num_ind = data.index('number')
                    if num_ind == ser_ind+1:
                        self.serial = data[num_ind+1]
                    else:
                        pass

                # Now, get the calibration coefficients
                for key in self.mo_coefficient_name_map.keys():
                    if key in data:
                        ind = data.index(key)
                        self.coefficients.update({self.mo_coefficient_name_map[key]: data[ind+1]})
                    else:
                        pass

            elif 'SBE 37 PRESSURE CALIBRATION DATA' in text[page_num]:
                # tokenize the text data and extract only key words
                tokens = word_tokenize(text[page_num])
                data = [word.lower() for word in tokens if not word in string.punctuation]

                # Now, find and record the calibration date
                if 'calibration' and 'date' in data:
                    cal_ind = data.index('calibration')
                    date_ind = data.index('date')
                    # Run check they are in order
                    if date_ind == cal_ind+1:
                        date = pd.to_datetime(data[date_ind+1]).strftime('%Y%m%d')
                        self.date.update({'PCAL': date})
                    else:
                        raise Exception(f"Can't locate pressure calibration date.")
                else:
                    raise Exception(f"Can't locate pressure calibration date.")

                # Check for the serial number
                if 'serial' and 'number' in data and len(self.serial) == 0:
                    ser_ind = data.index('serial')
                    num_ind = data.index('number')
                    if num_ind == ser_ind+1:
                        self.serial = data[num_ind+1]
                    else:
                        pass

                # Now, get the calibration coefficients
                for key in self.mo_coefficient_name_map.keys():
                    if key in data:
                        ind = data.index(key)
                        self.coefficients.update({self.mo_coefficient_name_map[key]: data[ind+1]})
                    else:
                        pass

            # Now check for other important information
            else:
                tokens = word_tokenize(text[page_num])
                data = [word.lower() for word in tokens if not word in string.punctuation]

                # Now, find the sensor rating
                if 'sensor' and 'rating' in data:
                    ind = data.index('rating')
                    self.coefficients.update({self.mo_coefficient_name_map['prange']: data[ind+1]})

    def read_cal(self, data):
        """
        Function which reads and parses the CTDBP calibration values stored
        in a .cal file.

        Args:
            filename - the name of the calibration (.cal) file to load. If the
                cal file is not located in the same directory as this script, the
                full filepath also needs to be specified.
        Returns:
            self.coefficients - populated coefficients dictionary
            self.date - the calibration dates associated with the calibration values
            self.type - the type (i.e. 16+/37-IM) of the CTD
            self.serial - populates the 5-digit serial number of the instrument
        """

        for line in data.splitlines():
            key, value = line.replace(" ", "").split('=')

            if key == 'INSTRUMENT_TYPE':
                if value == 'SEACATPLUS':
                    ctd_type = '16'
                elif value == '37SBE':
                    ctd_type = '37'
                else:
                    ctd_type = ''
                if self.ctd_type != ctd_type:
                    raise ValueError(f'CTD type in cal file {ctd_type} does not match the UID type {self.ctd_type}')

            elif key == 'SERIALNO':
                if self.serial != value.zfill(5):
                    raise Exception(f'Serial number {value.zfill(5)} stored in cal file does not match {self.serial} from the UID.')

            elif 'CALDATE' in key:
                self.date.update({key: datetime.datetime.strptime(value, '%d-%b-%y').strftime('%Y%m%d')})

            else:
                if self.ctd_type == '16':
                    name = self.coefficient_name_map.get(key)
                elif self.ctd_type == '37':
                    name = self.mo_coefficient_name_map.get(key)
                else:
                    pass

                if not name or name is None:
                    continue
                else:
                    self.coefficients.update({name: value})

    def load_cal(self, filepath):
        """
        Loads all of the calibration coefficients from the vendor cal files for
        a given CTD instrument class.

        Args:
            filepath - directory path to where the zipfiles are stored locally
        Raises:
            FileExistsError - Checks the given filepath that a .cal file exists
        Returns:
            self.coefficients - populated coefficients dictionary
            self.date - the calibration dates associated with the calibration values
            self.type - the type (i.e. 16+/37-IM) of the CTD
            self.serial - populates the 5-digit serial number of the instrument
        """

        if filepath.endswith('.zip'):
            with ZipFile(filepath) as zfile:
                filename = [name for name in zfile.namelist() if '.cal' in name]
                if len(filename) > 0:
                    data = zfile.read(filename[0]).decode('ASCII')
                    self.read_cal(data)
                else:
                    FileExistsError(f"No .cal file found in {filepath}.")

        elif filepath.endswith('.cal'):
            with open(filepath) as filename:
                data = filename.read()
                self.read_cal(data)

        else:
            FileExistsError(f"No .cal file found in {filepath}.")

    def read_xml(self, data):
        """
        Function which reads and parses the CTDBP calibration values stored
        in the xmlcon file.

        Args:
            data - the data string to parse
        Returns:
            self.coefficients - populated coefficients dictionary
            self.date - the calibration dates associated with the calibration values
            self.type - the type (i.e. 16+/37-IM) of the CTD
            self.serial - populates the 5-digit serial number of the instrument
        """

        Tflag  = False
        Cflag  = False
        O2flag = False

        for child in data.iter():
            key = child.tag.upper()
            value = child.text.upper()

            if key == 'NAME':
                if '16PLUS' in value:
                    ctd_type = '16'
                    if self.ctd_type != ctd_type:
                        raise ValueError(f'CTD type in xmlcon file {ctd_type} does not match the UID type {self.ctd_type}')

            # Check if we are processing the calibration values for the temperature sensor
            # If we already have parsed the Temp data, need to turn the flag off
            if key == 'TEMPERATURESENSOR':
                Tflag = True
            elif 'SENSOR' in key and Tflag == True:
                Tflag = False
            else:
                pass

            # Check on if we are now parsing the conductivity data
            if key == 'CONDUCTIVITYSENSOR':
                Cflag = True
            elif 'SENSOR' in key and Cflag == True:
                Cflag = False
            else:
                pass

            # Check if an oxygen sensor has been appended to the CTD configuration
            if key == 'OXYGENSENSOR':
                O2flag = True

            # Check that the serial number in the xmlcon file matches the serial
            # number from the UID
            if key == 'SERIALNUMBER':
                if self.serial != value.zfill(5):
                    raise Exception(f'Serial number {value.zfill(5)} stored in xmlcon file does not match {self.serial} from the UID.')

            # Parse the calibration dates of the different sensors
            if key == 'CALIBRATIONDATE':
                if Tflag:
                    self.date.update({'TCALDATE':datetime.datetime.strptime(value, '%d-%b-%y').strftime('%Y%m%d')})
                elif Cflag:
                    self.date.update({'CCALDATE': datetime.datetime.strptime(value, '%d-%b-%y').strftime('%Y%m%d')})
                else:
                    self.date.update({'PCALDATE': datetime.datetime.strptime(value, '%d-%b-%y').strftime('%Y%m%d')})

            # Now, we get to parse the actual calibration values, but it is necessary to make sure the
            # key names are correct
            if Tflag:
                key = 'T'+key

            name = self.coefficient_name_map.get(key)
            if not name or name is None:
                if O2flag:
                    name = self.o2_coefficients_map.get(key)
                    self.coefficients.update({name: value})
                else:
                    pass
            else:
                self.coefficients.update({name: value})

    def load_xml(self, filepath):
        """
        Loads all of the calibration coefficients from the vendor xmlcon files for
        a given CTD instrument class.

        Args:
            filepath - the name of the xmlcon file to load and parse. If the
                xmlcon file is not located in the same directory as this script,
                the full filepath also needs to be specified. May point to a zipfile.
        Raises:
            FileExistsError - Checks the given filepath that an xmlcon file exists
        Returns:
            self.coefficients - populated coefficients dictionary
            self.date - the calibration dates associated with the calibration values
            self.type - the type (i.e. 16+/37-IM) of the CTD
            self.serial - populates the 5-digit serial number of the instrument
        """

        if filepath.endswith('.zip'):
            with ZipFile(filepath) as zfile:
                filename = [name for name in zfile.namelist() if '.xmlcon' in name]
                if len(filename) > 0:
                    data = et.parse(zfile.open(filename[0]))
                    self.read_xml(data)
                else:
                    FileExistsError(f"No .cal file found in {filepath}.")

        elif filepath.endswith('.xmlcon'):
            with open(filepath) as file:
                data = et.parse(file)
                self.read_xml(data)

        else:
            FileExistsError(f"No .cal file found in {filepath}.")

    def load_qct(self, filepath):
        """
        Function which parses the output from the QCT check-in and loads them into
        the CTD object.

        Args:
            filepath - the full directory path and filename
        Raises:
            ValueError - checks if the serial number parsed from the UID matches the
                the serial number stored in the file.
        Returns:
            self.coefficients - populated coefficients dictionary
            self.date - the calibration dates associated with the calibration values
            self.type - the type (i.e. 16+/37-IM) of the CTD
            self.serial - populates the 5-digit serial number of the instrument
        """

        with open(filepath,errors='ignore') as filename:
            data = filename.read()

        if self.ctd_type == '37':
            data = data.replace('<', ' ').replace('>', ' ')
            for line in data.splitlines():
                keys = list(self.mo_coefficient_name_map.keys())
                if any([word for word in line.split() if word.lower() in keys]):
                    name = self.mo_coefficient_name_map.get(line.split()[0])
                    value = line.split()[-1]
                    self.coefficients.update({name: value})

        elif self.ctd_type == '16':
            if 'CTDBPP' in self.uid:
                # Okay, the CTDBPP (inductive mooring ones) have to be handled differently
                data = data.replace('<', ' ').replace('>', ' ')
                for line in data.splitlines():
                    keys = list(self.coefficient_name_map.keys())
                    
                    if any([word for word in line.split() if word in keys]):
                        name = self.coefficient_name_map.get(line.split()[0])
                        value = line.split()[1]
                        self.coefficients.update({name: value})
                        
                    elif 'SERIAL NO.' in line:
                        ind = line.split().index('NO.')
                        serial_num = line.split()[ind+1]
                        serial_num = serial_num.zfill(5)
                        if not self.serial == serial_num:
                            raise ValueError(f'UID serial number {self.serial} does not match the QCT serial num {serial_num}')
                    
                    elif 'CalDate' in line:
                        words = line.split()
                        self.date.update({str(len(self.date)): pd.to_datetime(words[1]).strftime('%Y%m%d')})
                    
                    else:
                        pass
            
            else:
                for line in data.splitlines():
                    keys = list(self.coefficient_name_map.keys())
                    if any([word for word in line.split() if word in keys]):
                        name = self.coefficient_name_map.get(line.split()[0])
                        value = line.split()[-1]
                        self.coefficients.update({name: value})

                    if 'temperature:' in line:
                        self.date.update({'TCAL': pd.to_datetime(line.split()[-1]).strftime('%Y%m%d')})
                    elif 'conductivity:' in line:
                        self.date.update({'CCAL': pd.to_datetime(line.split()[-1]).strftime('%Y%m%d')})
                    elif 'pressure S/N' in line:
                        self.date.update({'PCAL': pd.to_datetime(line.split()[-1]).strftime('%Y%m%d')})
                    else:
                        pass

                    if 'SERIAL NO.' in line:
                        ind = line.split().index('NO.')
                        serial_num = line.split()[ind+1]
                        serial_num = serial_num.zfill(5)
                        if not self.serial == serial_num:
                            raise ValueError(f'UID serial number {self.serial} does not match the QCT serial num {serial_num}')

                if 'SBE 16Plus' in line:
                    if self.ctd_type is not '16':
                        raise TypeError(f'CTD type {self.ctd_type} does not match the qct.')

        else:
            pass

    def write_csv(self, outpath):
        """
        This function writes the correctly named csv file for the ctd to the
        specified directory.

        Args:
            outpath - directory path of where to write the csv file
        Raises:
            ValueError - raised if the CTD object's coefficient dictionary
                has not been populated
        Returns:
            self.to_csv - a csv of the calibration coefficients which is
                written to the specified directory from the outpath.
        """

        # Run a check that the coefficients have actually been loaded
        if len(self.coefficients) == 0:
            raise ValueError('No calibration coefficients have been loaded.')

        # Create a dataframe to write to the csv
        data = {'serial': [self.ctd_type + '-' + self.serial]*len(self.coefficients),
                'name': list(self.coefficients.keys()),
                'value': list(self.coefficients.values()),
                'notes': ['']*len(self.coefficients)
                }
        df = pd.DataFrame().from_dict(data)

        # Generate the csv name
        cal_date = max(self.date.values())
        csv_name = self.uid + '__' + cal_date + '.csv'

        # Write the dataframe to a csv file
        # check = input(f"Write {csv_name} to {outpath}? [y/n]: ")
        check = 'y'
        if check.lower().strip() == 'y':
            df.to_csv(outpath+'/'+csv_name, index=False)